# Выбор локации для скважины

Допустим, вы работаете в добывающей компании «ГлавРосГосНефть». Нужно решить, где бурить новую скважину.

Вам предоставлены пробы нефти в трёх регионах: в каждом 10 000 месторождений, где измерили качество нефти и объём её запасов. Постройте модель машинного обучения, которая поможет определить регион, где добыча принесёт наибольшую прибыль. Проанализируйте возможную прибыль и риски техникой *Bootstrap.*

Шаги для выбора локации:

- В избранном регионе ищут месторождения, для каждого определяют значения признаков;
- Строят модель и оценивают объём запасов;
- Выбирают месторождения с самым высокими оценками значений. Количество месторождений зависит от бюджета компании и стоимости разработки одной скважины;
- Прибыль равна суммарной прибыли отобранных месторождений.

In [1]:
RS = 123
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error

## Загрузка и подготовка данных

----------
### Регион '0':

In [2]:
rg_0 = pd.read_csv('/datasets/geo_data_0.csv')
rg_0.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100000 entries, 0 to 99999
Data columns (total 5 columns):
id         100000 non-null object
f0         100000 non-null float64
f1         100000 non-null float64
f2         100000 non-null float64
product    100000 non-null float64
dtypes: float64(4), object(1)
memory usage: 3.8+ MB


In [3]:
print('Дубликатов строк:', rg_0.duplicated().sum())
print('Количество уникальных id:', rg_0['id'].nunique())
print('\nОписание числовых признаков:')
rg_0.describe().round(3)

Дубликатов строк: 0
Количество уникальных id: 99990

Описание числовых признаков:


,f0,f1,f2,product
count,100000.000,100000.000,100000.000,100000.000
mean,0.500,0.250,2.503,92.500
std,0.872,0.504,3.248,44.289
min,-1.409,-0.848,-12.088,0.000
25%,-0.073,-0.201,0.288,56.498
50%,0.502,0.250,2.516,91.850
75%,1.074,0.701,4.715,128.564
max,2.362,1.344,16.004,185.364


----------
### Регион '1':

In [4]:
rg_1 = pd.read_csv('/datasets/geo_data_1.csv')
rg_1.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100000 entries, 0 to 99999
Data columns (total 5 columns):
id         100000 non-null object
f0         100000 non-null float64
f1         100000 non-null float64
f2         100000 non-null float64
product    100000 non-null float64
dtypes: float64(4), object(1)
memory usage: 3.8+ MB


In [5]:
print('Дубликатов строк:', rg_1.duplicated().sum())
print('Количество уникальных id:', rg_1['id'].nunique())
print('\nОписание числовых признаков:')
rg_1.describe().round(3)

Дубликатов строк: 0
Количество уникальных id: 99996

Описание числовых признаков:


,f0,f1,f2,product
count,100000.000,100000.000,100000.000,100000.000
mean,1.141,-4.797,2.495,68.825
std,8.966,5.120,1.704,45.944
min,-31.610,-26.359,-0.018,0.000
25%,-6.299,-8.268,1.000,26.953
50%,1.153,-4.813,2.011,57.086
75%,8.621,-1.333,4.000,107.813
max,29.422,18.734,5.020,137.945


----------
### Регион '2':

In [6]:
rg_2 = pd.read_csv('/datasets/geo_data_2.csv')
rg_2.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100000 entries, 0 to 99999
Data columns (total 5 columns):
id         100000 non-null object
f0         100000 non-null float64
f1         100000 non-null float64
f2         100000 non-null float64
product    100000 non-null float64
dtypes: float64(4), object(1)
memory usage: 3.8+ MB


In [7]:
print('Дубликатов строк:', rg_2.duplicated().sum())
print('Количество уникальных id:', rg_2['id'].nunique())
print('\nОписание числовых признаков:')
rg_2.describe().round(3)

Дубликатов строк: 0
Количество уникальных id: 99996

Описание числовых признаков:


,f0,f1,f2,product
count,100000.000,100000.000,100000.000,100000.000
mean,0.002,-0.002,2.495,95.000
std,1.732,1.730,3.473,44.750
min,-8.760,-7.084,-11.970,0.000
25%,-1.162,-1.175,0.130,59.450
50%,0.009,-0.009,2.484,94.926
75%,1.159,1.164,4.859,130.595
max,7.238,7.845,16.739,190.030


### Вывод:  
Данные разных регионов имеют схожую структуру, пропуски данных и дубликаты строк отсутствуют.  
Идентификатор скважины для обучения модели не нужен, его следует удалить.

## Обучение и проверка модели

In [8]:
def split(df):
    train_df, valid_df = train_test_split(df, test_size = 0.25, random_state = RS)
    Y_train = train_df['product']
    X_train = train_df.drop(['id','product'], axis = 1)
    Y_valid = valid_df['product']
    X_valid = valid_df.drop(['id','product'], axis = 1)
    return Y_train, X_train, Y_valid, X_valid

Разделим данные каждого региона на обучающую и валидационные выборки в пропроции 75% - 25%   
и выделим признаки и цели.

In [9]:
Y_train_0, X_train_0, Y_valid_0, X_valid_0 = split(rg_0)
Y_train_1, X_train_1, Y_valid_1, X_valid_1 = split(rg_1)
Y_train_2, X_train_2, Y_valid_2, X_valid_2 = split(rg_2)

In [10]:
def create_model(Y_train, X_train, Y_valid, X_valid):
    model = LinearRegression()
    model.fit(X_train, Y_train)
    prediction = pd.Series(model.predict(X_valid))
    rmse = mean_squared_error(Y_valid,prediction)**0.5
    return prediction, rmse, model

Обучим модели на обучающих выборках   
и расчитаем среднеквадратичную ошибку на валидационных выборках.

In [11]:
predict_0, rmse_0, model_0 = create_model(Y_train_0, X_train_0, Y_valid_0, X_valid_0)
predict_1, rmse_1, model_1 = create_model(Y_train_1, X_train_1, Y_valid_1, X_valid_1)
predict_2, rmse_2, model_2 = create_model(Y_train_2, X_train_2, Y_valid_2, X_valid_2)

In [12]:
print('По предсказаниям моделей:')
print(f'Средний запас сырья в регионе 0: {predict_0.mean():0.3f} тыс.баррелей   RMSE: {rmse_0:0.3f}')
print(f'Средний запас сырья в регионе 1: {predict_1.mean():0.3f} тыс.баррелей   RMSE:  {rmse_1:0.3f}')
print(f'Средний запас сырья в регионе 2: {predict_2.mean():0.3f} тыс.баррелей   RMSE: {rmse_2:0.3f}')

По предсказаниям моделей:
Средний запас сырья в регионе 0: 92.549 тыс.баррелей   RMSE: 37.648
Средний запас сырья в регионе 1: 69.280 тыс.баррелей   RMSE:  0.895
Средний запас сырья в регионе 2: 95.099 тыс.баррелей   RMSE: 40.128


### Вывод :  
Наиболее точно модель линейной регрессии работает в регионе 1  
но при этом средний запас сырья в этом регионе существенно меньше

## Подготовка к расчёту прибыли

Константы для дальнейших расчетов:

In [13]:
PROFIT = 450_000 # 450 тыс. рублей за 1000 барелей
BUDGET = 10_000_000_000   # 10 млрд. рублей

In [14]:
print('Регион 0:', rg_0['product'].mean().round(3))
print('Регион 1:', rg_1['product'].mean().round(3))
print('Регион 2:', rg_2['product'].mean().round(3))

Регион 0: 92.5
Регион 1: 68.825
Регион 2: 95.0


In [15]:
print('Минимальный средний запас сырья в скважине для безубыточной работы:', 
      round(BUDGET / 200 / PROFIT, 3),
     'тыс.баррелей')

Минимальный средний запас сырья в скважине для безубыточной работы: 111.111 тыс.баррелей


### Вывод:   
Средний запас сырья во всех регионах недостаточен для безубыточной работы,  
требуется прогноз запасов и выбор лучших скважин

## Расчёт прибыли и рисков 

In [16]:
# Расчет фактической выручки с 200 скважин, лучших по прогнозу:
def total_return(target, predict):
    predict_sorted = predict.sort_values(ascending=False)
    selected = target[predict_sorted.index][:200]
    return PROFIT * selected.sum()

In [17]:
# Bootstrap на 1000 выборок по 500 скважин
state = np.random.RandomState(RS)
def profit_risk(target, predict, msg):
    print(msg)
    target = target.reset_index(drop=True)
    predict = predict.reset_index(drop=True)
    profits = []
    damages = []
    for i in range(1000):
        target_sample = target.sample(n = 500, replace = True, random_state = state)
        predict_sample = predict[target_sample.index]
        tot_ret = total_return(target_sample, predict_sample)
        profits.append(tot_ret - BUDGET)
        damages.append(tot_ret < BUDGET)
    
    profits = pd.Series(profits)
    damages = pd.Series(damages)
    
    print(f'Средняя прибыль {profits.mean()/1e6:0.1f} млн. рублей')
    confidence_interval = [round(profits.quantile(0.025)/1e6, 0), 
                           round(profits.quantile(0.975)/1e6, 0)] 
    print('Доверительный интервал 95%: ', confidence_interval, 'млн. рублей')
    print(f'Риски {damages.mean()*100:0.2f}%')

In [18]:
profit_risk(Y_valid_0, predict_0, 'Для региона 0:')

Для региона 0:
Средняя прибыль 504.8 млн. рублей
Доверительный интервал 95%:  [-52.0, 1030.0] млн. рублей
Риски 4.00%


In [19]:
profit_risk(Y_valid_1, predict_1, 'Для региона 1:')

Для региона 1:
Средняя прибыль 525.9 млн. рублей
Доверительный интервал 95%:  [100.0, 945.0] млн. рублей
Риски 0.90%


In [20]:
profit_risk(Y_valid_2, predict_2, 'Для региона 2:')

Для региона 2:
Средняя прибыль 387.0 млн. рублей
Доверительный интервал 95%:  [-174.0, 934.0] млн. рублей
Риски 7.70%


### Вывод:   
Только регион '1' имеет риск убытков менее 2.5%

## Общий вывод:   

<div class="alert alert-block alert-info">
    
<b>После оценки рисков регионов, в которых вероятность убытков меньше 2.5%  остается регион '1'  
    С вероятностью 95% прибыль в этом регионе составит от 100 до 945 млн. рублей,  
    средняя прибыль 526 млн. рублей, а вероятность получить убыток 0.9%  
    Кроме того, модель линейная регрессия работает наиболее точно в этом регионе
    </b>
</div>

